<h1>Speech to Text on Audio Files Automation
    <h4>Azure Cognitive Servisler içerisinde yer alan Speech servisi ile örnek bir sesten metne dönüşüm scripti.

Congitive Servisler Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/

Bu servislerin kullanımı için Azure Portal üzerinden bir servis oluşturulmalı, **Speech** .

**Speech Servisi**

Speech Servisleri Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/

Speech to Text Dökümantasyonu: https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/speech-to-text


In [ ]:
#pip install azure

In [ ]:
#pip install azure-cognitiveservices-speech

In [ ]:
#pip install azure-core

In [ ]:
from azure.core.credentials import AzureKeyCredential
import glob
import azure.cognitiveservices.speech as speechsdk
import time
import json
import pandas as pd
import csv

<h5>Bu örnekte 2 adet ses dosyası ile çalışmaktayız. Bu dosyaların wav formatında olması gerekiyor.
Ses dosyalarının ve output dosyasının ismi aşağıda tanımlanmalı. Output dosyası csv formatında oluşturulmuş olmalı.

In [ ]:
#
audio_input1 = speechsdk.AudioConfig(filename="audio1.wav") 

output1 = "output1"

In [ ]:
audio_input2 = speechsdk.AudioConfig(filename="audio2.wav") 
output2 = "output2"

<h5>Daha sonra oluşturduğumuz bu servislere ait bilgiler girilmeli.
    Bu bilgilere Resource içerisindeki Keys and Endpoint kısmından ulaşabiliriz.
 

In [ ]:
speech_key = "..."
speech_region = "..."

<h5>Speech to Text Fonksiyonları

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription= speech_key, region= speech_region) 

In [ ]:
def speech_recognize_continuous_from_file(file, output_file_name):
    
    speech_config = speechsdk.SpeechConfig(subscription="9fb7c3f3b6524358ad56915e0d8a6b60", region="westeurope") 
    audio_config = file
    
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config, language="tr-TR")
    
    done = False
    def stop_cb(evt):
        #print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True
    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)
    speech_recognizer.recognized.connect(handle_final_result)
    
    #Anlik speech to Text donusumunu takip etmek icin asagidaki satirlari uncomment ediniz.
    
    #speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    #speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    #speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    #speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    ##speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
    
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    print("Printing all results:")
    print(all_results)
    df = pd.DataFrame(all_results)
    df
    file_name = output_file_name + r".csv"
    df.to_csv(file_name)
    print ("Audio File: converted successfully")
    return all_results

In [ ]:
def speech_to_text(audio_input, output_file_name):
    print ("PROGRAM START")
    tic = time.perf_counter()
    
    file= audio_input

    # Loop to call function to convert audio files to text
    #run_speech_to_text_small_audio_files(file)
    all_results = speech_recognize_continuous_from_file(file, output_file_name)
    print(file)
    print ("PROGRAM END")
    toc = time.perf_counter()
    print(f"Converted speech to text in {toc - tic:0.4f} seconds")
    return all_results

<h3>AUDIO FILE 1
<h5>Speech To Text Dönüşümü

In [ ]:
stt_results1 = speech_to_text(audio_input1, output1)

<h3>AUDIO FILE 2
<h5>Speech To Text Dönüşümü

In [ ]:
stt_results2 = speech_to_text(audio_input2, output2)